```bash
# For loading the full model, all dependency groups that were installed during training should be installed now
pip install ./medicalmultitaskmodeling[api,detection,interactive] --extra-index https://gitlab.com/api/v4/groups/11254156/-/packages/pypi/simple
```

In [ ]:
import torch
import cv2
import torch.nn as nn
import torchvision.transforms.functional as F
from PIL import Image
from albumentations.augmentations.geometric.functional import resize
from mmm.labelstudio_ext.NativeBlocks import NativeBlocks, MMM_MODELS, DEFAULT_MODEL
from mmm.interactive import tasks
# We load the model from an archive,
# which allows us to load only those modules that are compatible with the current environment.
model = NativeBlocks(MMM_MODELS[DEFAULT_MODEL], device_identifier="cuda:0")
model.get_device(), model.get_task_keys()[:5], model.get_sharedblock_keys()

In [ ]:
import imageio.v3 as imageio
# Load an image from the web
raw_image = imageio.imread(r"https://owncloud.fraunhofer.de/index.php/s/n6gycdah9SaxOdD/download")
Image.fromarray(raw_image)

In [ ]:
# The input should be divisible by 32 and similar to the pre-training setting.
# For ImageNet, we trained with a static size of 256x256
input_image = resize(raw_image, height=256, width=256, interpolation=cv2.INTER_LINEAR)
# Inputs need to be between 0 and 1
input_image = F.to_tensor(input_image)

In [ ]:
with torch.inference_mode():
    # The model expects batches
    feature_pyramid = model["encoder"](input_image.unsqueeze(0).to(model.device))
    print([feature_map.shape for feature_map in feature_pyramid])
    hidden_vector = nn.Flatten(1)(model["squeezer"](feature_pyramid)[1])
    print(hidden_vector.shape)
    print(some_values := hidden_vector[0, :5].tolist())


Due to changes in hardware and version dependencies of the training environment, results may not be reproducible.
Here we check if the current environment can load the given model:

In [ ]:
differences = [
    x - y
    for x, y in zip(
        some_values,
        [-0.16809800267219543, 0.8937981724739075, 0.1215958520770073, 0.2989726662635803, 0.02798079140484333],
    )
]
if all(abs(difference) < 1e-6 for difference in differences):
    print(f"The reproducibility test passed! {differences=}")
else:
    raise Exception(f"The reproducibility test failed! {differences=}")


In addition, we can perform inference with one of the pretraining tasks. The ImageNet classifier should classify the image of a teddy bear correctly as "teddy, teddy bear"

In [ ]:
mtl_task: tasks.ClassificationTask = model['imgnetclf']
imagenet_classnames = mtl_task.class_names.copy()

# You can use the representation for classification. 
scores = nn.Softmax(dim=1)(mtl_task.task_modules["classification_head"](hidden_vector).detach().cpu())
# Print the top classes
print([(imagenet_classnames[i], scores[0, i].item()) for i in torch.argsort(scores, descending=True)[0, :5]])

In [ ]:
assert (highest_class := imagenet_classnames[torch.argmax(scores).item()]) == "teddy, teddy bear"